# Итоговый проект по предмету "Этика искусственного интеллекта"

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Импорт данных

In [2]:
data = pd.read_csv('data/data.csv')

In [3]:
data.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.000000,0.021277,0.872340,0.0000,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.000000,4,47
1,239607,0.912500,Yet call out all Muslims for the acts of a few...,0.050000,0.237500,0.612500,0.887500,0.1125,0.0,0.0,...,26670,approved,0,0,0,1,0,0.000000,4,80
2,239612,0.830769,This bitch is nuts. Who would read a book by a...,0.107692,0.661538,0.338462,0.830769,0.0000,0.0,0.0,...,26674,rejected,0,0,0,0,0,0.061538,4,65
3,240311,0.968750,You're an idiot.,0.031250,0.062500,0.000000,0.968750,0.0000,NaN,NaN,...,32846,rejected,0,0,0,0,0,0.000000,0,32
4,240329,0.900000,Who cares!? Stark trek and Star Wars fans are ...,0.100000,0.200000,0.000000,0.900000,0.0000,NaN,NaN,...,32846,rejected,0,0,0,0,0,0.300000,0,10


In [4]:
comments = data["comment_text"]
target = (data["target"] > 0.7).astype(int)

## Задание 1
### Разделение данных на train и test

In [5]:
X_train, X_test, y_train, y_test = train_test_split(comments, target, train_size=0.7)

In [6]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((63631,), (27271,), (63631,), (27271,))

## Задание 2

### Векторизация данных

In [7]:
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [8]:
X_train_vec

<63631x58188 sparse matrix of type '<class 'numpy.int64'>'
	with 2154674 stored elements in Compressed Sparse Row format>

## Задание 3

### Определение модели и её обучение

In [9]:
lr = LogisticRegression(random_state=42, max_iter=2000)
lr.fit(X_train_vec, y_train)
predict = lr.predict(X_test_vec)

In [10]:
print(f'Accuracy: {accuracy_score(predict, y_test) * 100:.2f}%')

Accuracy: 92.90%


## Задание 4

### Определение функции предсказания токсичности комментария

In [11]:
def predict_toxicity(comment, vectorizer, model):
    """
    This function predicts the toxicity of a given comment.

    Parameters:
    comment (str): The comment to be evaluated.
    vectorizer (CountVectorizer): The vectorizer object used for transforming the comment.
    model (LogisticRegression): The trained model used for making the prediction.

    Returns:
    float: The probability of the comment being toxic.
    """
        
    # Return the probability
    return model.predict_proba(vectorizer.transform([comment]))[:, 1][0]

In [12]:
comment = "ass"
toxicity_probability = predict_toxicity(comment, vectorizer, lr)
print(f"Comment toxicity: {toxicity_probability:.2f}")

Comment toxicity: 0.98


In [13]:
comment = "add"
toxicity_probability = predict_toxicity(comment, vectorizer, lr)
print(f"Comment toxicity: {toxicity_probability:.2f}")

Comment toxicity: 0.19


## Задание 5

### Проверка работы модели

In [14]:
text = input("Input some text and we'll try to predict it toxicity: ")

print(f"For comment '{text}' toxicity was: {predict_toxicity(text, vectorizer, lr):.2f}")

For comment 'I love machine learning' toxicity was: 0.10


## Задание 6

### Вывести 10 наиболее токсичных комментариев

In [15]:
# Create a dictionary of words and their coefficients, and sort it
toxic_words = sorted({word: cf for word, cf in zip(vectorizer.vocabulary_.keys(), lr.coef_[0])}.items(), key=lambda x: x[1], reverse=True)[:15]

# Print the words and their coefficients
for word, coefficient in toxic_words:
    print(f'Word: {word}, Coefficient: {coefficient}')

Word: nyb0, Coefficient: 9.18032249442941
Word: beaming, Coefficient: 8.698120619824167
Word: whatd, Coefficient: 8.616921856323954
Word: duality, Coefficient: 7.508836546355755
Word: lighthouse, Coefficient: 6.969421661464832
Word: lastly, Coefficient: 6.701959384633766
Word: harridan, Coefficient: 6.598331719342781
Word: hereditary, Coefficient: 6.5408938373967604
Word: molests, Coefficient: 6.48432678139763
Word: fleets, Coefficient: 6.376104619327502
Word: burgeoning, Coefficient: 6.330229892717647
Word: kahn, Coefficient: 6.274479675493844
Word: dashing, Coefficient: 6.2253906509131465
Word: gowdy, Coefficient: 5.850678692995031
Word: 627, Coefficient: 5.800983253648692


## Задание 7

При перезапуске блокнота коэффициенты могут измениться, привожу пример первого вывода:
Изначально, мне все комментарии показались неуместными, пришлось провести небольшой research.

- Hom: Это может быть сокращением или опечаткой оскорбительного термина, что может объяснить его высокий коэффициент.
- Impetus (импетус): Нейтральное слово, обычно используемое в формальных или академических контекстах. Его высокий уровень токсичности удивителен и может указывать на предвзятость или недостаток модели.
- Hanger (вешалка): Общее, не оскорбительное слово. Его связь с токсичностью может быть связана с конкретными контекстами, которые здесь не очевидны.
- Yamulkes (ярмулки): Слово, относящееся к религиозному предмету. Его высокий уровень токсичности может свидетельствовать о неправильном использовании в оскорбительных или дискриминационных контекстах, или может быть артефактом модели.
- Californian (калифорниец): Демоним; его присутствие здесь предполагает возможное использование в негативных или стереотипных контекстах.
- Clang (звон): В целом нейтральное слово, связанное со звуком. Его высокий балл может быть контекстуальным или аномальным.
- UAF: Акроним, который может иметь несколько значений. Без контекста трудно оценить, почему он считается токсичным.
- Pew (лавка): Нейтральное слово, часто связанное с сиденьями в церквях. Его токсичность может быть специфичной для контекста.
- Warden (начальник тюрьмы): Обычно связано с тюрьмами, его присутствие может быть связано с негативными контекстами в обсуждениях.
- Khalid: Личное имя, что предполагает, что его присутствие может быть связано с целенаправленными домогательствами или неправильным использованием.
- Headwaters (истоки): Нейтральный термин в географии или экологической науке. Его высокий балл неожидан и может указывать на аномалию модели.
- Hatriotism (ненавистный патриотизм): По-видимому, это словообразование от "ненависть" и "патриотизм". Этот неологизм, вероятно, имеет негативные коннотации, что объясняет его показатель токсичности.
- RAF: Акроним с различными значениями, включая "Королевские ВВС". Токсичность может зависеть от контекста.
- Dug (выкопанный): Прошедшее время от "копать". Нейтральное слово, его присутствие здесь необычно и может указывать на контекстуальное использование.
- Included (включенный): Общее, нейтральное слово. Его высокий уровень токсичности довольно удивителен и может указывать на ошибку моделирования.

В заключение, некоторые слова (например, "hatriotism" или потенциально оскорбительные термины) могут по своей сути появляться в токсичных контекстах, но многие другие (например, "impetus", "hanger", "pew") вероятно отражают контекстуальное использование или потенциальные аномалии в модели. Крайне важно пересмотреть и усовершенствовать модель, учитывая контекст использования и потенциальные предвзятости.

### Задание 8

Проверить как алгоритм работает на некоторых комментариях.

Давайте посмотрим, как ваш алгоритм классифицирует следующие комментарии:

"I have a christian friend"
"I have a muslim friend"
"I have a white friend"
"I have a black friend"

Что думаете о получившихся результатах? Есть ли у модели bias? Этичен ли он?

In [16]:
comments_list = ["I have a christian friend", "I have a muslim friend", "I have a white friend", "I have a black friend"]

In [17]:
for comment in comments_list:
    print(f"Токсичность комментария {comment}: {predict_toxicity(comment, vectorizer, lr):.2f}")

Токсичность комментария I have a christian friend: 0.19
Токсичность комментария I have a muslim friend: 0.52
Токсичность комментария I have a white friend: 0.40
Токсичность комментария I have a black friend: 0.60


### Выводы:

Результаты модели предсказания токсичности комментариев интересные и могут указывать на потенциальный смещенный (biased) характер модели. Рассмотрим каждый из результатов:

1) "I have a christian friend" (0.19): Это заявление имеет самый низкий показатель токсичности, что может отражать меньшую частоту негативных или стереотипных утверждений о христианах в данных, на которых обучалась модель.
2) "I have a muslim friend" (0.52): Более высокий уровень токсичности может быть связан с более высокой частотой стереотипных или негативных высказываний о мусульманах в обучающих данных.
3) "I have a white friend" (0.40) и "I have a black friend" (0.6): Заметно, что утверждение о черном друге имеет выше показатель токсичности, что может указывать на наличие предвзятости в данных, связанной с расовыми вопросами.

Эти результаты могут отражать смещенность (bias) в обучающих данных модели. Модели машинного обучения часто усваивают и воспроизводят существующие предрассудки и стереотипы из данных, на которых они обучаются. Если в обучающих данных преобладают негативные комментарии, связанные с определенными группами людей (например, определенные религии или расы), модель может неправильно интерпретировать нейтральные утверждения как токсичные.

По поводу этичности: использование такой модели может быть неэтичным, если она применяется в реальных условиях без должного понимания и коррекции ее предвзятости. Важно провести тщательный анализ данных и возможно переобучить модель на более сбалансированном наборе данных, чтобы уменьшить смещенность и улучшить точность предсказаний. Это включает в себя анализ не только самих текстов, но и контекста, в котором они используются.

## Задание 9

Я считаю, что я дал ответ на это задание выше.
Хотел бы только сделать ремарку про высказывание с платформы:

"Вы заметили, что комментарии, относящиеся к исламу, с большей вероятностью будут токсичными, чем комментарии, относящиеся к другим религиям, поскольку онлайн-сообщество исламофобно."

Это высказывание по моему не этично. Я тоже часть интернет сообщества и однако я не исламофоб, те же кто это писали, приравняли всё интернет сообщество к исламофобам, что является очень поверхностным суждением. Я считаю, что это высказывание не этично и не корректно.

### Задание 10

Я также считаю что я ответил на вопрос в прошлом задании, но дам более равномерный ответ тут.

Для улучшения алгоритма предсказания токсичности комментариев в направлении его этического применения, можно рассмотреть следующие идеи:

1. Использование более сбалансированных и разнообразных обучающих данных: Одна из основных проблем в работе с алгоритмами машинного обучения — это предвзятость в обучающих данных. Если данные содержат стереотипы или предубеждения, модель может воспроизводить эти же предубеждения. Решением может быть сбор данных из разнообразных источников и культур, что помогает уменьшить одностороннюю предвзятость. Также полезно включать данные, которые явно контрастируют с распространенными стереотипами, для обучения модели распознавать и отвергать предубеждения.

2. Контекстуальный анализ: Многие слова и фразы могут быть токсичными или не токсичными в зависимости от контекста. Развитие и применение алгоритмов, способных анализировать контекст, в котором используется язык, может значительно повысить точность и этичность модели. Это включает в себя улучшение понимания моделью намерений пользователя, семантических и культурных нюансов, а также различных форм использования языка (ирония, сарказм, шутки).

Применение этих подходов может помочь создать более справедливую, объективную и этичную систему, которая лучше понимает и интерпретирует человеческий язык в его многообразии.